In [ ]:
import pyarrow.dataset as ds
from graphique import GraphQL

format = ds.ParquetFileFormat(read_options={'dictionary_columns': ['state']})
dataset = ds.dataset('../tests/fixtures/zipcodes.parquet', format=format)
app = GraphQL(dataset)


def execute(query):
    result = app.schema.execute_sync(query, root_value=app.root_value)
    for error in result.errors or []:
        raise ValueError(error)
    return result.data

In [ ]:
execute(
    '''{
length
schema {
    names
    types
    index
}
}'''
)

In [ ]:
# California counties with the most cities
execute(
    '''{
filter(state: {eq: "CA"}) {
    group(by: "county", aggregate: {countDistinct: {name: "city", alias: "cities"}}) {
        sort(by: "-cities", length: 5) {
            columns {
                county {
                    values
                }
            }
            cities: column(name: "cities") {
                ... on LongColumn {
                    values
                }
            }
        }
    }
}
}'''
)